In [ ]:
from __future__ import print_function
import openvsp as vsp

import numpy as np
import os
import json

from scipy.spatial.transform import Rotation as R

import matplotlib.pyplot as plt

import pymeshlab

import pygem

import glob

import tqdm

import polyscope as ps

## Loading Meta Data

A json file is populated in the base data folder which includes the necessary information to process the data from openvsp. We will first load that json.

In [ ]:
f = open('./Base Data/Data_Breakdown.json')

meta_data = json.load(f)

f.close()

### Loading A Model

Now let's load the first model and save the mesh.

In [ ]:
vsp.ReadVSPFile('./Base Data/%s'%meta_data[0]['Model'])
vsp.ExportFile('test.obj',vsp.SET_ALL,vsp.EXPORT_OBJ)

print('Saved test.obj in the main folder oopen to see the model.')

If you open the model in meshlab the following is shown:

<img src="https://i.ibb.co/dfy0Q67/image.png" alt="image" border="0">

Now we will remove the test.obj file.

In [ ]:
os.remove('test.obj')

Now let's process the models into their components and create a labeled mesh (labeled by face color):

In [ ]:
def process_model(meta_data, fname = None):
    vsp.ClearVSPModel()
    if fname is None:
        fname = './Base Data/%s'%meta_data['Model']
    
    vsp.ReadVSPFile(fname)
    vsp.Update()
    #Save each geometry seprately
    for i in range(meta_data['sets'] + 1):
        vsp.ExportFile('temp-%i.obj'%i,3+i,vsp.EXPORT_OBJ)
    
    ms = pymeshlab.MeshSet()

    for part in ['Fuselage','Stabalizers','Engine','Wing']:
        inds = meta_data['Parts'][part]

        ms = pymeshlab.MeshSet()

        for i in inds:
            ms.load_new_mesh('temp-%i.obj'%i)
            try:
                ms.meshing_close_holes(maxholesize = 10000)
            except:
                pass
            
        if ms.number_meshes()>1:
            ms.generate_boolean_union(first_mesh=0,second_mesh=1)

            for i in range(2,len(inds)):
                ms.generate_boolean_union(first_mesh=i,second_mesh=ms.number_meshes()-1)

        if len(inds)>0:
            ms.set_current_mesh(ms.number_meshes()-1)
            ms.save_current_mesh(part+'.obj')

    ms = pymeshlab.MeshSet()
    ms2 = pymeshlab.MeshSet()  

    c = 0
    colors = [[1.0,0.0,0.0,1.0],[0.0,1.0,0.0,1.0],[0.0,0.0,1.0,1.0],[0.0,1.0,1.0,1.0]]
    for part in ['Fuselage','Stabalizers','Engine','Wing']:
        inds = meta_data['Parts'][part]

        if len(inds)>0:
            ms2.load_new_mesh(part+'.obj')

            m = ms2.current_mesh()
            
            color = np.zeros([m.face_number(),4]) + [colors[c]]

            mm = pymeshlab.Mesh(vertex_matrix = m.vertex_matrix(), 
                                face_matrix = m.face_matrix(), 
                                edge_matrix=m.edge_matrix(),
                                f_color_matrix =color
                                )
            ms.add_mesh(mesh=mm)
            
            try:
                ms.meshing_close_holes(maxholesize = 10000)
            except:
                pass
            os.remove(part+'.obj')
        c += 1

    n_parts = ms.number_meshes()

    ms.generate_boolean_union(first_mesh=0,second_mesh=1,transfer_face_color = True)

    for i in range(2,n_parts):
        ms.generate_boolean_union(first_mesh=i,second_mesh=ms.number_meshes()-1,transfer_face_color = True)
    
    m = ms.current_mesh()
    
    if meta_data['Orientation'][0] == '+z' and meta_data['Orientation'][1] == '+x':
        v_mat = np.copy(m.vertex_matrix())
        v_mat = v_mat - v_mat.min(0)
        v_mat = v_mat/v_mat.max()
        v_mat = v_mat + (1 - v_mat.max(0))/2
        mm = pymeshlab.Mesh(vertex_matrix = v_mat, 
                                face_matrix = m.face_matrix(), 
                                edge_matrix=m.edge_matrix(),
                                f_color_matrix =m.face_color_matrix()
                                )
    else:
        rot = R.from_rotvec(-np.pi/2 * np.array([0, 1, 0])).as_matrix()
        v_mat = (rot @ np.copy(m.vertex_matrix()).T).T
        v_mat = v_mat - v_mat.min(0)
        v_mat = v_mat/v_mat.max()
        v_mat = v_mat + (1 - v_mat.max(0))/2
        mm = pymeshlab.Mesh(vertex_matrix = v_mat, 
                                face_matrix = m.face_matrix(), 
                                edge_matrix=m.edge_matrix(),
                                f_color_matrix =m.face_color_matrix()
                                )
    
    ms.add_mesh(mm)
    ms.set_current_mesh(ms.number_meshes()-1)
    ms.save_current_mesh(fname.replace('vsp3','obj').split('/')[-1])

    for i in range(meta_data['sets'] + 1):
        os.remove('temp-%i.obj'%i)

In [ ]:
for m in meta_data:
    process_model(m)

Now if you open any of the saved obj files in mesh lab with face color enabled you will see:

<img src="https://i.ibb.co/PZcYJD8/image.png" alt="image" border="0" width="100%">

## Augmenting the Data

Now that we have established how to process the data into useful models we shall now implement some basic augmentation approaches to increase the size of the dataset from 10 models to much more.

### Augmenting By Design Paramter Purturbation
Let's now start by purturbing the design parameters for each model based on manually determined ranges in the meta-data json file to create design variations.

In [ ]:
def get_random_variations(meta_data,n = 20):
    
    vsp.ClearVSPModel()
    
    fname = meta_data['Model']
    
    vsp.ReadVSPFile('./Base Data/%s'%fname)

    parms = []
    # First grab all the parameters
    for parm in meta_data['Params']:
        g = vsp.FindGeomsWithName(parm['Geom'])[0]
        p = vsp.FindParm(g,parm['Name'],parm['GroupName'])
        parms.append([p,parm['Range'],vsp.GetParmVal(p)])

    
    for i in range(n):
        for p in parms:
            vsp.SetParmVal(p[0],np.clip(np.random.normal(loc=p[2], scale= (p[1][1]-p[1][0])/6),p[1][0],p[1][1]))
            
        vsp.Update()
        
        vsp.WriteVSPFile('./Augmented/' + fname.replace('.','_%i.'%(i+1)))
        
    for i in range(n):        
        process_model(meta_data,fname = './Augmented/' + fname.replace('.','_%i.'%(i+1)))

In [ ]:
for m in meta_data:
    get_random_variations(m)

### Augmenting By Remeshing
Let's first start by remehsing the meshes using isotropic explicit re meshing to get more uniform mesh faces and control their size to represent each design with different mesh structures.

In [ ]:
mesh_files = glob.glob("./*.obj")

for mesh_file in mesh_files:
    
    for i in range(4):     
        ms = pymeshlab.MeshSet()
        ms.load_new_mesh(mesh_file)
        ms.meshing_isotropic_explicit_remeshing(targetlen =pymeshlab.Percentage((i + 2) * 0.2) , maxsurfdist = pymeshlab.Percentage(0.5))
        ms.save_current_mesh(mesh_file.replace('.obj','_rmesh_%i.obj'%(i+1)))

### Saving The Meshes For Easier Access In Training

Now let's save our dataset in numpy format and ready for graph based learning.

In [ ]:
mesh_files = glob.glob("./*.obj")
mesh_files.sort()
data_set = []

for mesh_file in tqdm.tqdm(mesh_files):
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(mesh_file)
    
    m = ms.current_mesh()
    
    v_mat = m.vertex_matrix().astype(np.float32)
    f_mat = m.face_matrix().astype(np.int32)
    edges = []

    for f in f_mat:
        edges.append([f[0],f[1]])
        edges.append([f[1],f[0]])
        edges.append([f[0],f[2]])
        edges.append([f[2],f[0]])
        edges.append([f[1],f[2]])
        edges.append([f[2],f[1]])

    edges = np.array(edges).astype(np.int32)
    edges = np.unique(edges,axis=0)

    label_mat = m.face_color_matrix()
    labels = np.zeros([f_mat.shape[0],4])
    labels[np.where(label_mat[:,0])[0],0] = 1.0
    labels[np.where(np.logical_and(label_mat[:,1],np.logical_not(label_mat[:,2])))[0],1] = 1.0
    labels[np.where(np.logical_and(label_mat[:,2],np.logical_not(label_mat[:,1])))[0],2] = 1.0
    labels[np.where(np.logical_and(label_mat[:,2],label_mat[:,1]))[0],3] = 1.0

    labels = labels.astype(bool)
    
    data_set.append([v_mat,f_mat,edges,labels])

data_set = np.array(data_set)

np.save('dataset.npy',data_set)

### Active Augmentation Pipeline
Now we will create an active augmentation pipeline which will be used to randomly alter the models during training. Four primary approaches are introduced here:

<ul>
    <li><b>Free Form Deformation</b>: We randomly apply FFD to any given model before training.</li>
    <li><b>Random Scaling</b>: The scale of the model will be altered randomly +/- 20 %.</li>
    <li><b>Random Rotation</b>: The models will be randomly rotated +/- 45 degrees across all three axes.</li>
    <li><b>Random Translation</b>: The models will be randomly moved in space from the center of a unit box maximum translation in each direction will be limited to +/- 0.15</li>
</ul>

Let's start with a simple sphere and see how the random augmentation  works

In [ ]:
def mesh_points(num_pts = 2000):
    indices = np.arange(0, num_pts, dtype=float) + 0.5

    phi = np.arccos(1 - 2*indices/num_pts)
    theta = np.pi * (1 + 5**0.5) * indices

    return (np.array([np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)]).T + 1.0)/2.0

mesh = mesh_points()
plt.figure(figsize=(8,8)).add_subplot(111, projection='3d').scatter(*mesh.T);
plt.show()

In [ ]:
def Augmentation_pipeline(v_mat,ffd_scale = 0.35, scale=0.1, rotation=np.pi/12,translation=0.0):
    
    ffd = pygem.FFD([4, 4, 4])
    ffd.array_mu_x = np.random.uniform(low = -ffd_scale, high=ffd_scale , size=ffd.array_mu_x.shape)
    ffd.array_mu_y = np.random.uniform(low = -ffd_scale, high=ffd_scale , size=ffd.array_mu_y.shape)
    ffd.array_mu_z = np.random.uniform(low = -ffd_scale, high=ffd_scale , size=ffd.array_mu_z.shape)
    
    v_mat = ffd(v_mat)
    
    v_mat = (R.from_rotvec(np.random.uniform(low=-rotation,high=rotation,size=3)).as_matrix() @ v_mat.T).T
    
    v_mat *= (1-np.random.uniform(high=scale))
    
    v_mat = v_mat - v_mat.min(0)
    v_mat = v_mat + (1 - v_mat.max(0))/2
    
    v_mat += np.random.uniform(low=-translation,high=translation,size=3)
    
    
    
    return v_mat

In [ ]:
new_mesh = Augmentation_pipeline(mesh)

ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')
ax.scatter(*new_mesh.T)
ax.scatter(*mesh.T)
plt.show()

This Augmentaion will be applied randomly to all samples during training.

### Rendering OBJ Files
Now we will implement rendering and save the images of our dataset

In [ ]:
def render_mesh(file,save_file ='render.png', view='isometric',shadow=False,colorize=True, def_color=[0.7,0.7,0.7]):
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(file)
    m = ms.current_mesh()
        
    v_mat = m.vertex_matrix().astype(np.float32)
    f_mat = m.face_matrix().astype(np.int32)
    label_mat = m.face_color_matrix()

    ps.set_autocenter_structures(True)
    ps.set_autoscale_structures(True)
    ps.init()

    if colorize:
        ps_mesh = ps.register_surface_mesh("my mesh", v_mat, f_mat[np.where(label_mat[:,0])[0]],color=[1.0,0.0,0.0])
        ps_mesh1 = ps.register_surface_mesh("my mesh 1", v_mat, f_mat[np.where(np.logical_and(label_mat[:,1],np.logical_not(label_mat[:,2])))[0]],color=[0.0,1.0,0.0])
        ps_mesh2 = ps.register_surface_mesh("my mesh 2", v_mat, f_mat[np.where(np.logical_and(label_mat[:,2],np.logical_not(label_mat[:,1])))[0]],color=[0.0,0.0,1.0])
        ps_mesh3 = ps.register_surface_mesh("my mesh 3", v_mat, f_mat[np.where(np.logical_and(label_mat[:,2],label_mat[:,1]))[0]],color=[0.0,1.0,1.0])
    else:
        ps_mesh = ps.register_surface_mesh("my mesh", v_mat, f_mat,color=def_color)

    ps.set_ground_plane_height_factor(0.08)
    ps.set_up_dir("z_up")

    if shadow:
        ps.set_ground_plane_mode("shadow_only")
    else:
        ps.set_ground_plane_mode("none")


    if view == 'isometric':
        ps.look_at((-1.0,-1.0,1.0),(.0, .0, .0))
    elif view == 'top':
        ps.look_at((-0.0,-0.001,1.8),(.0, .0, .0))
    elif view == 'bottom':
        ps.look_at((-0.0,-0.001,-1.8),(.0, .0, .0))
    elif view == 'left':
        ps.look_at((-0.0,-1.8,0.0),(.0, .0, .0))
    elif view == 'right':
        ps.look_at((-0.0,1.8,0.0),(.0, .0, .0))
    elif view == 'front':
        ps.look_at((-1.8,0.0,0.0),(.0, .0, .0))
    elif view == 'back':
        ps.look_at((1.8,0.0,0.0),(.0, .0, .0))

    ps.screenshot(filename=save_file)
    # ps.show()


In [ ]:
mesh_files = glob.glob(".\*.obj")

for mesh_file in tqdm.tqdm(mesh_files):
    render_mesh(mesh_file,'./isometric_view/' + mesh_file.split('\\')[-1].replace('.obj','.png'))
    render_mesh(mesh_file,'./top_view/' + mesh_file.split('\\')[-1].replace('.obj','.png'),view='top')
    render_mesh(mesh_file,'./bottom_view/' + mesh_file.split('\\')[-1].replace('.obj','.png'),view='bottom')
    render_mesh(mesh_file,'./left_view/' + mesh_file.split('\\')[-1].replace('.obj','.png'),view='left')
    render_mesh(mesh_file,'./right_view/' + mesh_file.split('\\')[-1].replace('.obj','.png'),view='right')
    render_mesh(mesh_file,'./front_view/' + mesh_file.split('\\')[-1].replace('.obj','.png'),view='front')
    render_mesh(mesh_file,'./back_view/' + mesh_file.split('\\')[-1].replace('.obj','.png'),view='back')